# US Immigration trends
### Data Engineering Capstone Project

#### Project Summary
In this capstone project, I used immigration dataset from 2016 to study immigration patterns into the US and answer following questions
 * where do immigrants come from 
 * what cities do immigrants go

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import logging
import numpy as np
import pandas as pd
import psycopg2
from pprint import pprint
from collections import defaultdict
import configparser
import psycopg2
import pandas as pd
import re

from datetime import datetime, timedelta
import logging
from sqlalchemy import create_engine


In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s  [%(name)s] %(message)s')
LOG = logging.getLogger('immig_etl')

### Step 1: Scope the Project and Gather Data

#### Scope 
Immigration dataset from 2016 and city demographics dataset is used to study immigration patterns into the US. Used immigration data from US National Tourism and Trade office and City demographic data to understand these trends.  Used Pandas library to read and load data into Redshift. SQL and python scripts were created to build the ETL.

#### Data source used and description
* Immigration data - This data comes from the US National Tourism and Trade Office. https://travel.trade.gov/research/reports/i94/historical/2016.html
* US City demographic data - Data source is opensoft. https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/
* Airport code - Table of airport codes and cities. https://datahub.io/core/airport-codes#data 
* I94_SAS_Labels_Descriptions - Sas file provided gives a better understanding of the data in immigration dataset.

#### Data description and data gathering
Before loading the data into a Redshift db, I will do some exploratory data analysis here to find out what cleaning steps must be performed and understand the DDL I must define for my dim and fact tables.


### Step 2: Explore and Assess the Data
#### Data exploration 
## Immigration data exploration

In [3]:
immigration_data_fnames = ['../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat',
                           '../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat']

In [4]:

filename = immigration_data_fnames[0]
iterator = pd.read_sas(
    filename, 'sas7bdat', encoding='ISO-8859-1', chunksize=20
)
idf = next(iterator)
print(idf.shape)
idf.head()

(20, 28)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,...,NaN,NaN,1996.0,D/S,M,NaN,LH,346608285.0,424,F1
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,...,NaN,NaN,1996.0,D/S,M,NaN,LH,346627585.0,424,F1
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,...,NaN,M,1999.0,07152016,F,NaN,AF,381092385.0,338,B2
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,...,NaN,M,1971.0,07152016,F,NaN,AF,381087885.0,338,B2
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,...,NaN,M,2004.0,07152016,M,NaN,AF,381078685.0,338,B2


In [5]:
dfs = []
for fname in immigration_data_fnames:
    files = pd.read_sas(fname, 'sas7bdat', encoding='ISO-8859-1', chunksize=20)
    dfs.append(next(files))
    continue
# print(dfs[0])    

In [5]:
with open('data/I94_SAS_Labels_Descriptions.SAS') as f:
    #readFile = f.read()
    #print(readFile)
    f.seek(0)
    lines = f.readlines()
comments = [line for line in lines if line.startswith('/*') and line.endswith('*/\n')]
print(comments)

['/* I94YR - 4 digit year */\n', '/* I94MON - Numeric month */\n', '/* I94CIT & I94RES - This format shows all the valid and invalid codes for processing */\n', '/* I94PORT - This format shows all the valid and invalid codes for processing */\n', '/* I94MODE - There are missing values as well as not reported (9) */\n', '/* I94BIR - Age of Respondent in Years */\n', '/* COUNT - Used for summary statistics */\n', '/* DTADFILE - Character Date Field - Date added to I-94 Files - CIC does not use */\n', '/* VISAPOST - Department of State where where Visa was issued - CIC does not use */\n', '/* OCCUP - Occupation that will be performed in U.S. - CIC does not use */\n', '/* ENTDEPA - Arrival Flag - admitted or paroled into the U.S. - CIC does not use */\n', '/* ENTDEPD - Departure Flag - Departed, lost I-94 or is deceased - CIC does not use */\n', '/* ENTDEPU - Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use */\n', '/* MATFLAG - Match flag - Match 

#### Column descriptions
Understanding the columns in Immigration dataset.
* cicid - primary key
* i94yr - arrival year
* i94mon - arrival month
* i94cit - citizenship
* i94res - country of residence
* i94port - arrival port
* arrdate - arrival date
* i94mode - mode of arrival land, sea, air
* i94addr - arrival address
* depdate - departure date
* i94bir - age of immigrant in years
* i94visa - visa type: business, pleasure, student
* count - "used for summary statistics" 
* dtadfile - character date field CIC does not use
* visapost - Department of State where visa was issued - CIC does not use
* occup - occupation that will be performed. CIC does not use
* entdepd - Arrival Flag - admitted or paroled into the U.S. - CIC does not use
* entdepu - Departure Flag - Departed, lost I-94 or is deceased - CIC does not use
* matflag - match of arrival and departure records
* biryear - year of birth
* dtaddto - max date of stay - CIC does not use
* gender - gender
* insnum - INS number
* airline - airline of arrival
* admnum - admission number
* fltno - flight number of arrival flight
* visatype - class of admission legally admitting non-immigrant to temporarily stay in US

In [6]:
zipped = zip(immigration_data_fnames, dfs)
# list(zipped)

In [7]:
cnames_month = {z[0].split('/')[-1].split('_')[1][:3]: list(z[1].columns.values) for z in zipped}

In [8]:
print(len(cnames_month['jan']))
for k, v in cnames_month.items():
    if (len(cnames_month[k]) != 28):
        print(k)
        #print(v)
        print(len(v))

28
jun
34


Month of June has more 6 olumns than other months.
#### Cleaning Steps
Let's identify the columns that are not in other months and clean up the data.

In [9]:
jan = list(dfs[0].columns.values)
jun = list(dfs[5].columns.values)

for z in (zip(jan, jun)):
    print(z)

('cicid', 'cicid')
('i94yr', 'i94yr')
('i94mon', 'i94mon')
('i94cit', 'i94cit')
('i94res', 'i94res')
('i94port', 'i94port')
('arrdate', 'arrdate')
('i94mode', 'i94mode')
('i94addr', 'i94addr')
('depdate', 'depdate')
('i94bir', 'i94bir')
('i94visa', 'i94visa')
('count', 'count')
('dtadfile', 'validres')
('visapost', 'delete_days')
('occup', 'delete_mexl')
('entdepa', 'delete_dup')
('entdepd', 'delete_visa')
('entdepu', 'delete_recdup')
('matflag', 'dtadfile')
('biryear', 'visapost')
('dtaddto', 'occup')
('gender', 'entdepa')
('insnum', 'entdepd')
('airline', 'entdepu')
('admnum', 'matflag')
('fltno', 'biryear')
('visatype', 'dtaddto')


We can see that June has 'validres' and 5 other columns that begin with delete_. When immigration data is loaded into fact tables skip these 6 columns.

## Temperature data exploration.

In [10]:
temperature_df = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
print(temperature_df.shape)
temperature_df.head()

(8599212, 7)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


## City demographics data exploration

In [11]:
city_demo_df = pd.read_csv('data/us-cities-demographics.csv', delimiter=';')
print(city_demo_df.shape)
city_demo_df.head()

(2891, 12)


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


Let's look at one city Newark to understand data partition. We see that eatch row is at City/State/Race level.

In [12]:
city_demo_df[city_demo_df['City'] == 'Newark'].head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
1770,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,Black or African-American,144961
1967,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,Asian,7349
2168,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,American Indian and Alaska Native,2268
2580,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,Hispanic or Latino,100432


Let's check if city data needs cleaning up. Things to check include
* Male and Female population adds up to Total population in this dataset.
* Foreign born, Number of verterans populations are less than total population.
* Distinct states should be close to 50. DC and Puerto Rico are the other entities besides states

In [15]:
# Confirm city data looks clean.
for _, row in city_demo_df.iterrows():
    if pd.notnull(row['Foreign-born']):
        assert row['Foreign-born'] <= row['Total Population']
    if pd.notnull(row['Male Population']):
        assert row['Male Population'] + row['Female Population'] == row['Total Population']
    if pd.notnull(row['Number of Veterans']):
        assert row['Number of Veterans'] <= row['Total Population']
    if pd.notnull(row['Count']):
        assert row['Count'] <= row['Total Population']
    for coll in ['Male Population', 'Female Population', 'Total Population', 'Number of Veterans', 'Foreign-born', 'Count']:
        assert pd.isnull(row[coll]) or row[coll] > 0

City dataset looks clean. Let's look at state data and do some sanity tests. We should expect CA, TX should be in the top 5 most populated states. The number of states should be at most 50 + DC and PR.

In [16]:
state_df = city_demo_df[['State Code', 'Male Population', 'Female Population', 'Total Population', 'Foreign-born']]
state_df = state_df.groupby(['State Code']).sum().reset_index()
print(state_df.shape)
state_df.sort_values(by=['Total Population'], ascending=False).head() #we have 49 rows and CA, TX and NY are in top 3.

(49, 5)


,State Code,Male Population,Female Population,Total Population,Foreign-born
4,CA,61055672.0,62388681.0,123444353,37059662.0
44,TX,34862194.0,35691659.0,70553853,14498054.0
34,NY,23422799.0,25579256.0,49002055,17186873.0
9,FL,15461937.0,16626425.0,32306132,7845566.0
14,IL,10943864.0,11570526.0,22514390,4632600.0


## Airport data exploration
To understand airport data
* Check the data size.
* Number of foreign airports.
* Do we have airports with coordinates that is NULL.
* Find out if all continent lengths are 2.

In [17]:
airport_codes_df = pd.read_csv('data/airport-codes_csv.csv')
print(airport_codes_df.shape)
airport_codes_df.head()


(55075, 12)


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [18]:
# Count foreign airports
airport_codes_df[airport_codes_df['iso_country'] != 'US'].shape

(32318, 12)

In [19]:
# Check if we have any coordinates that is NULL
airport_codes_df['coordinates'].isnull().values.any()
# Are all continents length 2 ?
airport_codes_df[airport_codes_df['continent'].str.len() > 2]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Since the scope of the project is to understand immigration events, our data model will be a star schema with immigration data as the fact table.  Immigration data will be loaded into a table, data will be explored, validated and cleaned after it is loaded. We will then create several dimension tables around our fact immigration data which will be detailed below. To ensure my data model is reproducible I will be creating fact and dim tables in python code. See following python files in this workspace.
* sql_queries.py - has data model of fact and dim tables.
* create_tables.py - Creates dim and fact tables in Redshift.
* etl.py - Builds the pipeline to load the data.

First, here is the data model for immigration data. Mirrors the columns in the immigration data set.

Dimension table _**d_arrivalmode**_
* Foreign key is fact_immigration.i94mode

Dimension table _**d_visatype**_
* Foreign key is fact_immigration.i94visa

Dimension table _**d_address**_
* Foreign key is fact_immigration.i94addr

Dimension table _**d_port**_

Foreign key is fact_immigration.i94port

Dimension table _**d_country**_

Dimension table _**d_city**_

Dimension table _**d_state**_
* Data aggregated from city table explained in data exploration.

#### 3.2 Mapping Out Data Pipelines

Following steps will be performed to map out data pipelines. All data will be imported from python file **etl.py** in this workspace. Reason being we want this etl load process to be reproducible and re-runnable.
* Manually insert dimenstion arrival modes. This has 3 distinct modes.
* Manually insert dimension visa type. This has 4 distinct visa types.
* Populate dimension address by parsing out lines from header file.
* Populate dimension port table by parsing out lines from header file.
* Populate dimension country table by parsing out lines from header file.
* Populate dimension city table by reading data from us-cities-demographics.csv
* Populate dimension state table by simply aggregating state data from us-cities-demographics.csv
* Populate immigration fact table by reading 12 files from immigration data frame. Here we have to be careful importing June data and not import the 6 columns we identified in out data exploration.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

Build the data pipelines to create the data model. See **sql_queries.py** and **create_tables.py** for dropping and creating dimension and fact tables.
Run the following command to create table from terminal window. _**python create_tables.py**_

### Create the data pipeline from python scripts
Run the following python scripts to create the pipeline. I used jupyter notebook to design and check the scripts, but highly recommend running python scripts to re-create the pipeline.

* Create a REDSHIFT instance in AWS, Use StartupRedshift.ipynb as a template. Make sure your settings are cofigured in dwh.cfg
* _**python create_tables.py**_
* _**python etl.py**_

In [9]:
import configparser
import psycopg2
from sql_queries import create_dim_table_queries, drop_dim_table_queries, create_fact_table_queries, drop_fact_table_queries
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [10]:
# SQL queries to drop dimension and fact tables.
drop_arrivalmode = 'DROP TABLE IF EXISTS d_arrivalmode;'
drop_visatype = 'DROP TABLE IF EXISTS d_visatype;'
drop_date = 'DROP TABLE IF EXISTS d_date;'
drop_address = 'DROP TABLE IF EXISTS d_address;'
drop_port = 'DROP TABLE IF EXISTS d_port;'
drop_country = 'DROP TABLE IF EXISTS d_country;'
drop_city = 'DROP TABLE IF EXISTS d_city;'
drop_state = 'DROP TABLE IF EXISTS d_state;'
drop_immig_fact = 'DROP TABLE IF EXISTS fact_immigration;'
drop_temp = 'DROP TABLE IF EXISTS d_temperature;'

In [11]:
# Drop all dimension tables
drop_dim_table_queries = [drop_arrivalmode, drop_visatype, drop_date, drop_address, drop_port, drop_country, drop_city, drop_state, drop_temp]
for query in drop_dim_table_queries:
    cur.execute(query)
    conn.commit()

In [12]:
# Drop fact table
drop_fact_table_queries = [drop_immig_fact ]
for query in drop_fact_table_queries:
    cur.execute(query)
    conn.commit()

In [13]:
# CREATE DIMENSION and FACT tables
create_arrivalmode = """CREATE TABLE IF NOT EXISTS d_arrivalmode
(id INT, mode CHAR(12));"""

create_visatype = """CREATE TABLE IF NOT EXISTS d_visatype
(id INT, visa_type CHAR(8));"""

create_address = 'CREATE TABLE IF NOT EXISTS d_address (id CHAR(5), name VARCHAR);'

create_port = 'CREATE TABLE IF NOT EXISTS d_port (id CHAR(5), name VARCHAR);'

create_country = """CREATE TABLE IF NOT EXISTS d_country
(id INT PRIMARY KEY, name VARCHAR NOT NULL);"""

create_city = """CREATE TABLE IF NOT EXISTS d_city
(city VARCHAR, state VARCHAR, median_age numeric, male_pop INT, female_pop INT, total_pop INT, num_vets INT,
foreign_born INT, avg_household_size FLOAT, state_code CHAR(2), race VARCHAR, count INT);
"""

create_state = """CREATE TABLE IF NOT EXISTS d_state
(state_code CHAR(2) PRIMARY KEY, male_pop INT, female_pop INT, total_pop INT, foreign_born INT);"""

create_date = """CREATE TABLE d_date
(id INT PRIMARY KEY, year INT NOT NULL, month INT NOT NULL,
 day INT NOT NULL, day_of_week INT NOT NULL, ymd_dash CHAR(10) NOT NULL,
 ymd_nodash CHAR(8) NOT NULL, mdy_nodash CHAR(8) NOT NULL);
"""

create_immig_fact = """CREATE TABLE IF NOT EXISTS fact_immigration 
(immigration_id INTEGER IDENTITY(0,1) PRIMARY KEY,
 cicid INT NOT NULL,
 i94yr INT NOT NULL,
 i94mon INT NOT NULL,
 i94cit INT,
 i94res INT,
 i94port CHAR(5),
 arrdate INT,
 i94mode INT,
 i94addr CHAR(5),
 depdate INT,
 i94bir INT,
 i94visa INT,
 count INT,
 dtadfile VARCHAR,
 visapost CHAR(5),
 occup CHAR(5),
 entdepa CHAR(1),
 entdepd CHAR(1),
 entdepu CHAR(1),
 matflag CHAR(1),
 biryear INT,
 dtaddto VARCHAR,
 gender CHAR(1),
 insnum VARCHAR,
 airline CHAR(3),
 admnum VARCHAR,
 fltno VARCHAR,
 visatype CHAR(3)
);
"""

create_temp = """CREATE TABLE IF NOT EXISTS d_temperature (
    dt date, avg_temp numeric, avg_temp_uncertainty numeric, city varchar,
    country varchar, latitude varchar, longitude varchar
);"""

In [40]:
create_dim_table_queries = [create_arrivalmode, create_visatype, create_date, create_address, create_port, create_country, create_city, create_state, create_temp]
for query in create_dim_table_queries:
    cur.execute(query)
    conn.commit()

In [46]:
create_fact_table_queries = [create_immig_fact]
for query in create_fact_table_queries:
    cur.execute(query)
    conn.commit()

In [58]:
# Populate visa type dimension table
LOG.info('Populating visa type')
insert_visatype = ("""INSERT INTO d_visatype (id, visa_type) VALUES (1, 'Business'), (2, 'Pleasure'), (3, 'Student');""")
cur.execute(insert_visatype)
conn.commit()
LOG.info('Finished inserting visa type')

2020-01-28 22:38:48,562 INFO  [immig_etl] Populating visa type
2020-01-28 22:38:52,323 INFO  [immig_etl] Finished inserting visa type


In [60]:
# Populate arrival mode dimension table
LOG.info('Populating arrival mode dim table')
insert_arrivalmode = ("""INSERT INTO d_arrivalmode (id, mode) VALUES (1, 'Air'), (2, 'Sea'), (3, 'Land'), (9, 'Not reported');""")
cur.execute(insert_arrivalmode)
conn.commit
LOG.info('Finshed populating arrival mode dim table')

2020-01-28 22:40:52,286 INFO  [immig_etl] Populating arrival mode dim table
2020-01-28 22:40:55,804 INFO  [immig_etl] Finshed populating arrival mode dim table


In [65]:
# Populate address dimension by reading header file lines and parsing out the address lines.
LOG.info('Populating address dim table')
insert_address = 'INSERT INTO d_address (id, name) VALUES (%s, %s);'

with open('data/I94_SAS_Labels_Descriptions.SAS') as f:
    desc_lines = f.readlines()

address_lines = desc_lines[981:1036]
regex = re.compile(r"^\s*'(?P<id>..)'\s*=\s*'(?P<name>.+)'.*$")
addresses = [regex.match(line) for line in address_lines]
adds = {a.group('id'): a.group('name') for a in addresses}
assert len(adds) == len(address_lines)

for it in sorted(adds.items()):
    cur.execute(insert_address, list(it))
LOG.info('Finished populating address dim table')

2020-01-28 22:43:24,340 INFO  [immig_etl] Populating address dim table
2020-01-28 22:43:34,361 INFO  [immig_etl] Finished populating address dim table


In [70]:
# Populate port dimension table
LOG.info('Populating arrival port dim table')
query = 'INSERT INTO d_port (id, name) VALUES (%s, %s);'

with open('data/I94_SAS_Labels_Descriptions.SAS') as f:
    desc_lines = f.readlines()

port_lines = desc_lines[302:962]        
regex = re.compile(r"^\s*'(?P<id>...?)'\s*=\s*'(?P<name>.+)'.*$")
ports = [regex.match(line) for line in port_lines]
allports = {p.group('id'): p.group('name').strip() for p in ports}
assert len(allports) == len(port_lines)

for it in sorted(allports.items()):
    cur.execute(query, list(it))
LOG.info('Finished populating arrival port dim table')

2020-01-28 22:59:19,966 INFO  [immig_etl] Populating arrival port dim table
2020-01-28 23:00:12,727 INFO  [immig_etl] Finished populating arrival port dim table


In [76]:
# Populate city dimension table
LOG.info('Populating city dim table')
insert_city = ("""INSERT INTO d_city
(city, state, median_age, male_pop, female_pop, total_pop, num_vets, foreign_born, avg_household_size, state_code, race, count)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""")

city_df = pd.read_csv('data/us-cities-demographics.csv', delimiter=';')
for _, row in city_df.iterrows():
    cur.execute(insert_city, [city if pd.notna(city) else None for city in row])

LOG.info('Finished populating city dim table')

2020-01-28 23:51:06,269 INFO  [immig_etl] Populating city dim table
2020-01-28 23:57:04,979 INFO  [immig_etl] Finished populating city dim table


In [78]:
# Populate state dimension table.
LOG.info('Populating state dim table')
insert_state = """INSERT INTO d_state (state_code, male_pop, female_pop, total_pop, foreign_born)
                  VALUES (%s, %s, %s, %s, %s);"""

city_df = pd.read_csv('data/us-cities-demographics.csv', delimiter=';')
state_df = city_df[['State Code', 'Male Population', 'Female Population', 'Total Population', 'Foreign-born']].groupby(['State Code']).sum().reset_index()

for _, row in state_df.iterrows():
    cur.execute(insert_state, list(row))
    conn.commit
LOG.info('Finished populating state dim table')

2020-01-28 23:57:47,114 INFO  [immig_etl] Populating state dim table
2020-01-28 23:57:55,428 INFO  [immig_etl] Finished populating state dim table


In [83]:
#Populating country dimension table
LOG.info('Populating country dim table')
query = """INSERT INTO d_country (id, name) VALUES (%s, %s);"""

with open('data/I94_SAS_Labels_Descriptions.SAS') as f:
    desc_lines = f.readlines()

country_lines = desc_lines[9:298]
regex = re.compile(r"^\s*(?P<id>\d+)\s*=\s*'(?P<name>.+)'.*$")
countries = [regex.match(l) for l in country_lines]
allcountries = {int(c.group('id')): c.group('name') for c in countries}
assert len(allcountries) == len(country_lines)

for it in sorted(allcountries.items()):
    cur.execute(query, list(it))

LOG.info('Finished populating country dim table')

2020-01-29 00:02:25,604 INFO  [immig_etl] Populating country dim table
2020-01-29 00:02:54,408 INFO  [immig_etl] Finished populating country dim table


In [ ]:
#Populating temperature dimension table.
LOG.info('Populating temperature dim table')
insert_temp = ("""INSERT INTO d_temperature(dt, avg_temp, avg_temp_uncertainty, city, country, latitude, longitude)
                 VALUES (%s, %s, %s, %s, %s, %s, %s);""")

temperature_df = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
for _, row in temperature_df.iterrows():
    cur.execute(insert_temp, [t if pd.notna(t) else None for t in row])
    conn.commit
LOG.info('Finished populating temperature dim table')

2020-01-29 03:55:39,512 INFO  [immig_etl] Populating temperature dim table


#### 4.2 Data Quality Checks
Confirm dimension tables are populated and have the expected rows.


Confirm the fact_immigration table is populated

Birth year must be reasonable. Human life expectancy must be limited to 125.

#### 4.3 Data dictionary 

## Step 5: Summary

### Tools and Technologies
* Tools used are Amazom Redshift database. I used a relational database to make querying my fact and dimensional tables easy and gain analytics on immigration patterns. Immigration data was well structured, that I was able to use a relational Db.
* Chose Redshift because it is fast, scalable and highly performant.
* ETL itself was written using Python, and pandas. This allowed me to explore the data, understand and come up with a _star schema_ with immigration fact table and several dimension tables.
* By creating city, state, country dimension tables, we are able to see the states immigrants are immigrating to and it also helps us understand the demographics of the cities and states they are headed to.

### Update frequency
* fact_immigration must be updated monthly when each new dataset is available.
* Dim tables can be dropped and recreated completely whenever fact_immigration is updated.
* dim_city can be simply updated whenever city's metadata changes or inserted when a city is not found.
* dim_state can be simply updated whenever city's metadata changes. this is an aggregate.

### If data was increased 100x
* If data was increased 100x, I would use Redshift spectrum. Here SQL queries can be performed on data that is stored in Amazon S3 buckets.
* This can save time and money because it eliminates the need to move data from a storage service to a database, and instead directly queries data inside an S3 bucket. 
* Redshift Spectrum also expands the scope of a given query because it extends beyond a user's existing Redshift data warehouse nodes and into large volumes of unstructured S3 data lakes.

REF: 
* https://searchaws.techtarget.com/definition/Amazon-Redshift-Spectrum
* https://aws.amazon.com/blogs/big-data/amazon-redshift-spectrum-extends-data-warehousing-out-to-exabytes-no-loading-required/

### If dashboard must be updated by 7am daily
* I would build a concurrent data orchestration pipeline using Amazon EMR and Apache Spark. An Airflow DAG that has an S3 sensor could read data as soon as it arrives, parse the data and land it in S3 location immediately. Our Reshift spectrum detailed above could read the data immediately to populate the dashboard.

REF: 
* https://aws.amazon.com/blogs/big-data/build-a-concurrent-data-orchestration-pipeline-using-amazon-emr-and-apache-livy/
* https://aws.amazon.com/blogs/big-data/amazon-redshift-spectrum-extends-data-warehousing-out-to-exabytes-no-loading-required/

### If database needed to be accessed by 100+ people
* AWS Redshift spectrum with date partitioned data should perform well for multiple users.
* We could have data replicated to different nodes, if our users are global we could have replication nodes closer to user's location.
